## Imports ##

In [39]:
import pandas as pd
import torch
import numpy as np
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

In [40]:
#Loading Dataset
DATASET_PATH = "../Datasets/Covid_Cleaned.csv"
COVID_data = pd.read_csv(DATASET_PATH)

## Batch preperation ##

In [41]:
X = COVID_data.drop(columns=['CLASIFFICATION_FINAL'])  # features (20)
y = COVID_data['CLASIFFICATION_FINAL']                 # target (low / high risk of COVID-19)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=32) # 70% train, 30% test
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=32) # test -> 50% validation, 50% final test

In [42]:
# Creating Pytorch Tensors
X_train_tensor = torch.tensor(X_train.to_numpy(), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.float32)

X_val_tensor = torch.tensor(X_val.to_numpy(), dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.to_numpy(), dtype=torch.float32)

X_test_tensor = torch.tensor(X_test.to_numpy(), dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.float32)

In [43]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

In [44]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Model Training ##

In [45]:
import torch.nn as nn
import torch.optim as optim
from src.utils import OurModel


In [46]:
ml = OurModel()

In [47]:
# model configuration
config = nn.Sequential(
    nn.Linear(20, 20),
    nn.ReLU(),
    nn.Linear(20, 20),
    nn.Tanh(),
    nn.Linear(20, 20),
    nn.ReLU(),
    nn.Linear(20, 1),
)
ml.add_configuration(config)

In [48]:
criterion = nn.BCELoss()
optimizer = optim.RMSprop(ml.parameters(), lr=0.001)

In [13]:
#Model Training

criterion = nn.BCEWithLogitsLoss()

for input, true in train_loader:
    true = true.unsqueeze(1)
    y_pred = ml(input)
    loss = criterion(y_pred, true)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [38]:
# Model Training
criterion = nn.BCEWithLogitsLoss()

epochs = 20
for epoch in range(epochs):
    ml.train()  # set model to training mode
    total_loss = 0
    correct = 0
    total = 0

    for inputs, true in train_loader:
        # 1. Match tensor shapes
        true = true.unsqueeze(1)

        # 2. Forward pass
        y_pred = ml(inputs)

        # 3. Compute loss
        loss = criterion(y_pred, true)

        # 4. Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 5. Collect training stats
        total_loss += loss.item()
        preds = torch.sigmoid(y_pred)              # convert logits → probabilities
        predicted = (preds > 0.5).float()          # threshold to 0/1
        correct += (predicted == true).sum().item()
        total += true.size(0)

    # 6. Epoch summary
    avg_loss = total_loss / len(train_loader)
    accuracy = correct / total

    print(f"Epoch [{epoch+1}/{epochs}] | Loss: {avg_loss:.4f} | Accuracy: {accuracy:.4f}")


Epoch [1/20] | Loss: 0.6367 | Accuracy: 0.6306
Epoch [2/20] | Loss: 0.6366 | Accuracy: 0.6310
Epoch [3/20] | Loss: 0.6364 | Accuracy: 0.6315
Epoch [4/20] | Loss: 0.6363 | Accuracy: 0.6312
Epoch [5/20] | Loss: 0.6363 | Accuracy: 0.6311
Epoch [6/20] | Loss: 0.6361 | Accuracy: 0.6317
Epoch [7/20] | Loss: 0.6360 | Accuracy: 0.6318
Epoch [8/20] | Loss: 0.6359 | Accuracy: 0.6312
Epoch [9/20] | Loss: 0.6356 | Accuracy: 0.6314
Epoch [10/20] | Loss: 0.6356 | Accuracy: 0.6328
Epoch [11/20] | Loss: 0.6355 | Accuracy: 0.6326
Epoch [12/20] | Loss: 0.6353 | Accuracy: 0.6328
Epoch [13/20] | Loss: 0.6353 | Accuracy: 0.6322
Epoch [14/20] | Loss: 0.6352 | Accuracy: 0.6331
Epoch [15/20] | Loss: 0.6351 | Accuracy: 0.6327
Epoch [16/20] | Loss: 0.6349 | Accuracy: 0.6332
Epoch [17/20] | Loss: 0.6351 | Accuracy: 0.6323
Epoch [18/20] | Loss: 0.6349 | Accuracy: 0.6328
Epoch [19/20] | Loss: 0.6349 | Accuracy: 0.6330
Epoch [20/20] | Loss: 0.6346 | Accuracy: 0.6338


In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(ml.parameters(), lr=1e-4)

for epoch in range(20):
    total_loss = 0
    for inputs, true in train_loader:
        true = true.unsqueeze(1)
        inputs = inputs.float()
        true = true.float()
        y_pred = ml(inputs)
        loss = criterion(y_pred, true)
        if torch.isnan(loss):
            print("NaN detected -> breaking")
            break
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch [{epoch+1}/20] | Loss: {total_loss/len(train_loader):.4f}")


Epoch [1/20] | Loss: 0.6660
Epoch [2/20] | Loss: 0.6427
Epoch [3/20] | Loss: 0.6401
